In [3]:
# pip install opencv-python numpy
import cv2
import os
import numpy as np

# Define ellipse detection, translation, and rotation function
def detect_ellipses_and_align(input_folder, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all files in input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.png'):  # Process only PNG files
            input_image_path = os.path.join(input_folder, file_name)
            output_image_path = os.path.join(output_folder, os.path.splitext(file_name)[0] + '.png')

            # Load image and convert to grayscale
            img_gray = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
            if img_gray is None:
                print(f"Cannot read image: {input_image_path}")
                continue

            # Load original color image (for ellipse rendering)
            img_color = cv2.imread(input_image_path, cv2.IMREAD_COLOR)
            if img_color is None:
                print(f"Cannot read image: {input_image_path}")
                continue

            # Binary thresholding
            _, binary = cv2.threshold(img_gray, 1, 255, cv2.THRESH_BINARY)

            # Contour detection
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Create blank mask, initialized as pure black
            mask = np.zeros_like(img_gray)

            # Initialize variables
            ellipse = None

            # Iterate through each contour and fit ellipse
            for contour in contours:
                # Filter out contours that are too small
                if cv2.contourArea(contour) < 500:  # Assume minimum area is 500
                    continue

                # Draw qualifying contours on mask (white)
                cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

                # Check if ellipse fitting is possible
                if len(contour) >= 5:  # At least 5 points required for ellipse fitting
                    ellipse = cv2.fitEllipse(contour)  # Fit ellipse
                    break  # Assume only one main ellipse per image

            if ellipse is None:
                print(f"No ellipse detected: {input_image_path}")
                continue

            # Use mask to filter out non-qualifying pixels
            img_cleaned = cv2.bitwise_and(img_color, img_color, mask=mask)

            # Extract ellipse parameters
            ((cx, cy), (major_axis, minor_axis), angle) = ellipse

            # Create 750x750 pure black background image, manually determined, adjustable based on actual cuttlefish size, corresponding coordinates below should be adjusted accordingly
            black_background = np.zeros((750, 750, 3), dtype=np.uint8)

            # Calculate target position (center ellipse at background image center)
            target_center = (375, 375)  # Background image center
            translation_matrix = np.float32([[1, 0, target_center[0] - cx], [0, 1, target_center[1] - cy]])

            # Translate image
            translated_img = cv2.warpAffine(img_cleaned, translation_matrix, (750, 750))

            # Calculate rotation angle (make major axis vertical)
            rotation_angle = -(180 - angle)  # Make major axis change from horizontal to vertical, 180 is angle, adjustable based on actual needs (0 or 180)
            rotation_matrix = cv2.getRotationMatrix2D(target_center, rotation_angle, 1.0)

            # Rotate image
            aligned_img = cv2.warpAffine(translated_img, rotation_matrix, (750, 750))

            # Overlay rotated image onto black background
            mask_aligned = aligned_img > 0  # Create mask to extract non-zero pixels
            black_background[mask_aligned] = aligned_img[mask_aligned]

            # Crop image to 500x500 pixels
            cropped_img = black_background[125:625, 125:625]  # Crop center region

            # Save result to output folder
            cv2.imwrite(output_image_path, cropped_img)
            print(f"Saved aligned image: {output_image_path}")

# Set input and output folder paths
input_folder = 'C:/Users/12152/Desktop/data/sepia4/vision_white_after/segmentation_cut_test'  # Folder 1 path
output_folder = 'C:/Users/12152/Desktop/data/sepia4/vision_white_after/segmentation_cut_alignment_test'  # Output folder path

# Call ellipse detection and alignment function
detect_ellipses_and_align(input_folder, output_folder)


SyntaxError: invalid syntax (1446855707.py, line 1)